# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import timedelta
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Format today and yesterday into strings for filename creation
format = "%m-%d-%Y"
now = datetime.now()
today = now.strftime(format)
yesterday = (now - timedelta(days=1)).strftime(format)

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    jh_live_date = today
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    jh_live_date = yesterday
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
print(jh_live_date)

12-03-2022


In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2022-12-04 04:20:56,33.93911,67.709953,206145,7834,NaN,NaN,Afghanistan,529.549924,3.800238
1,NaN,NaN,NaN,Albania,2022-12-04 04:20:56,41.15330,20.168300,333391,3594,NaN,NaN,Albania,11584.925985,1.078014
2,NaN,NaN,NaN,Algeria,2022-12-04 04:20:56,28.03390,1.659600,271102,6881,NaN,NaN,Algeria,618.233870,2.538159
3,NaN,NaN,NaN,Andorra,2022-12-04 04:20:56,42.50630,1.521800,47219,157,NaN,NaN,Andorra,61113.052482,0.332493
4,NaN,NaN,NaN,Angola,2022-12-04 04:20:56,-11.20270,17.873900,104676,1924,NaN,NaN,Angola,318.490679,1.838053


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/24/22,11/25/22,11/26/22,11/27/22,11/28/22,11/29/22,11/30/22,12/1/22,12/2/22,12/3/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,205506,205541,205612,205612,205802,205830,205907,206073,206133,206145
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,333305,333316,333322,333330,333330,333338,333343,333360,333381,333391
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,271050,271057,271061,271061,271079,271082,271090,271096,271100,271102


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/24/22,11/25/22,11/26/22,11/27/22,11/28/22,11/29/22,11/30/22,12/1/22,12/2/22,12/3/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7833,7833,7833,7833,7833,7833,7833,7834,7834,7834
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3594,3594,3594,3594,3594,3594,3594,3594,3594,3594
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6881,6881,6881,6881,6881,6881,6881,6881,6881,6881


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/24/22,11/25/22,11/26/22,11/27/22,11/28/22,11/29/22,11/30/22,12/1/22,12/2/22,12/3/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/24/22,11/25/22,11/26/22,11/27/22,11/28/22,11/29/22,11/30/22,12/1/22,12/2/22,12/3/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,18612,18612,18612,18612,18612,18612,18680,18680,18680,18680
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,66480,66480,66480,66480,66480,66480,66730,66730,66730,66730
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,6968,6968,6968,6968,6968,6968,6980,6980,6980,6980


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/24/22,11/25/22,11/26/22,11/27/22,11/28/22,11/29/22,11/30/22,12/1/22,12/2/22,12/3/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,230,230,230,230,230,230,230,230,230,230
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,716,716,716,716,716,716,716,716,716,716
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,103,103,103,103,103,103,103,103,103,103


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
678,1001.0,Autauga,Alabama,US,2022-12-04 04:20:56,32.539527,-86.644082,18680,230,NaN,NaN,"Autauga, Alabama, US",33435.357712,1.231263
715,1075.0,Lamar,Alabama,US,2022-12-04 04:20:56,33.779950,-88.096680,4524,69,NaN,NaN,"Lamar, Alabama, US",32770.735241,1.525199
716,1077.0,Lauderdale,Alabama,US,2022-12-04 04:20:56,34.901719,-87.656247,28777,411,NaN,NaN,"Lauderdale, Alabama, US",31033.441534,1.428224


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,98967906,1081431,0.0
India,44674390,530627,0.0
France,38174893,160133,0.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,98967906,1081431,0.0,97886475.0,2022-12-04 04:20:56,37.934269,-91.444083
India,44674390,530627,0.0,44143763.0,2022-12-04 04:20:56,23.088275,81.806127
France,38174893,160133,0.0,38014760.0,2022-12-04 04:20:56,6.430808,-34.730285
Germany,36557861,158198,0.0,36399663.0,2022-12-04 04:20:56,51.571844,10.277881
Brazil,35337546,690074,0.0,34647472.0,2022-12-04 04:20:56,-12.669522,-48.480493


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,11543461,97617,0.0
Texas,8105643,91423,0.0
Florida,7270410,83201,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,11543461,97617,0.0,11445844.0,2022-12-04 04:20:56,37.843962,-120.728594
Texas,8105643,91423,0.0,8014220.0,2022-12-04 04:20:56,31.660643,-98.653069
Florida,7270410,83201,0.0,7187209.0,2022-12-04 04:20:56,28.940755,-82.700744
New York,6401124,74050,0.0,6327074.0,2022-12-04 04:20:56,42.544151,-75.474183
Illinois,3885398,40339,0.0,3845059.0,2022-12-04 04:20:56,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,3552019,34213,0.0
Florida,Miami-Dade,1482343,11920,0.0
Arizona,Maricopa,1461222,18125,0.0
Illinois,Cook,1454935,14960,0.0
Texas,Harris,1226840,11381,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,3552019,34213,0.0,3517806.0,2022-12-04 04:20:56,34.308284,-118.228241,6037.0
Florida,Miami-Dade,1482343,11920,0.0,1470423.0,2022-12-04 04:20:56,25.611236,-80.551706,12086.0
Arizona,Maricopa,1461222,18125,0.0,1443097.0,2022-12-04 04:20:56,33.348359,-112.491815,4013.0
Illinois,Cook,1454935,14960,0.0,1439975.0,2022-12-04 04:20:56,41.841448,-87.816588,17031.0
Texas,Harris,1226840,11381,0.0,1215459.0,2022-12-04 04:20:56,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-12-01,206073,333360,271096,47219,104676,11,9106,9727247,445737,10742127,...,993875,246674,11981,547631,11516489,703036,535,11945,333746,259164
2022-12-02,206133,333381,271100,47219,104676,11,9106,9727247,445737,10748145,...,993875,246833,11981,547631,11517124,703036,535,11945,333746,259164
2022-12-03,206145,333391,271102,47219,104676,11,9106,9727247,445737,10751318,...,993875,246833,11981,547772,11517517,703036,535,11945,333746,259164


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-12-01,7834,3594,6881,157,1924,0,146,130025,8710,16203,...,7537,1637,14,5828,43176,5708,0,2159,4019,5620
2022-12-02,7834,3594,6881,157,1924,0,146,130025,8710,16244,...,7537,1637,14,5828,43176,5708,0,2159,4019,5620
2022-12-03,7834,3594,6881,157,1924,0,146,130025,8710,16244,...,7537,1637,14,5829,43177,5708,0,2159,4019,5620


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-12-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-12-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-12-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/24/22,11/25/22,11/26/22,11/27/22,11/28/22,11/29/22,11/30/22,12/1/22,12/2/22,12/3/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,18612,18612,18612,18612,18612,18612,18680,18680,18680,18680
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,4514,4514,4514,4514,4514,4514,4524,4524,4524,4524
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,28685,28685,28685,28685,28685,28685,28777,28777,28777,28777
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,9001,9001,9001,9001,9001,9001,9015,9015,9015,9015
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,44613,44613,44613,44613,44613,44613,44730,44730,44730,44730


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-12-01,1549285,299869,8263,2337547,969642,11533193,1709688,927641,317174,49,...,2389250,8104999,1058874,146777,23536,2151313,1859858,615040,1927880,180925
2022-12-02,1549285,299869,8263,2337547,970396,11543461,1711047,928712,317405,49,...,2389250,8105643,1063268,146777,23545,2153223,1859858,615332,1929325,180925
2022-12-03,1549285,299869,8263,2337547,970857,11543461,1711047,929249,317405,49,...,2389250,8105643,1063268,146777,23545,2153223,1859858,615332,1929325,180925


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2022-12-01       18680   66730    6980  7637  17500    2853   6102   39001   
2022-12-02       18680   66730    6980  7637  17500    2853   6102   39001   
2022-12-03       18680   66730    6980  7637  17500    2853   6102   39001   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-12-01        10155     6262  ...    7564   2225     9585     2273   
2022-12-02        10155     6262  ...    7564   2225     9585     2273   
2022-12-03        10155     6262  ...    7564   2225     9585     2273   

Province_State                                                     
Admin2         Sweetwater  Teton Uinta Unassigned Washakie Weston  
2022-12-01          12304  11911  6255          0     2681   1860  
2022-12-02          12304  11911  6255          0     2681   1860  
2022-12-03          12304  11911  6255          0     2681   1860  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-12-01,20652,1436,34,31751,12571,97603,13627,11587,3172,0,...,28305,91423,5110,770,125,22437,14705,7618,15619,1938
2022-12-02,20652,1436,34,31751,12578,97617,13632,11587,3172,0,...,28305,91423,5127,770,125,22457,14705,7627,15624,1938
2022-12-03,20652,1436,34,31751,12585,97617,13632,11587,3172,0,...,28305,91423,5127,770,125,22457,14705,7627,15624,1938


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-12-01         230     716     103  108    259      54    130     661   
2022-12-02         230     716     103  108    259      54    130     661   
2022-12-03         230     716     103  108    259      54    130     661   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-12-01          169       87  ...     151     43       84       28   
2022-12-02          169       87  ...     151     43       84       28   
2022-12-03          169       87  ...     151     43       84       28   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-12-01            134    16    43          0       47     22  
2022-12-02            134    16    43          0       47     22  
2022-12-03            134    16    43          0       47     22  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change().round(7)
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-12-01,0.000806,0.000051,0.000022,0.0,0.0,0.0,0.0,0.0,0.0,0.007714,...,0.0,0.000625,0.0,0.000429,0.000051,0.0,0.0,0.0,0.0,0.0
2022-12-02,0.000291,0.000063,0.000015,0.0,0.0,0.0,0.0,0.0,0.0,0.000560,...,0.0,0.000645,0.0,0.000000,0.000055,0.0,0.0,0.0,0.0,0.0
2022-12-03,0.000058,0.000030,0.000007,0.0,0.0,0.0,0.0,0.0,0.0,0.000295,...,0.0,0.000000,0.0,0.000258,0.000034,0.0,0.0,0.0,0.0,0.0


In [60]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [61]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change().round(7)
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-12-01,0.000128,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.00409,...,0.0,0.0,0.0,0.000000,0.000023,0.0,NaN,0.0,0.0,0.0
2022-12-02,0.000000,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.00253,...,0.0,0.0,0.0,0.000000,0.000000,0.0,NaN,0.0,0.0,0.0
2022-12-03,0.000000,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.000172,0.000023,0.0,NaN,0.0,0.0,0.0


In [62]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [63]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change().round(7)
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [65]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change().round(7)
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-12-01,0.0,0.0,0.0,0.0,0.000796,0.000785,0.000834,0.000749,0.000688,0.0,...,0.0,0.000255,0.00000,0.0,0.000680,0.000838,0.001189,0.000641,0.000749,0.0
2022-12-02,0.0,0.0,0.0,0.0,0.000778,0.000890,0.000795,0.001154,0.000728,0.0,...,0.0,0.000079,0.00415,0.0,0.000382,0.000888,0.000000,0.000475,0.000749,0.0
2022-12-03,0.0,0.0,0.0,0.0,0.000475,0.000000,0.000000,0.000578,0.000000,0.0,...,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [66]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [67]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change().round(7)
jh_hist_uscounties_cases_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-12-01         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-12-02         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-12-03         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-12-01          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2022-12-02          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2022-12-03          0.0      0.0  ...     0.0    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-12-01            0.0   0.0   0.0        NaN      0.0    0.0  
2022-12-02            0.0   0.0   0.0        NaN      0.0    0.0  
2022-12-03            0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [68]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [69]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change().round(7)
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-12-01,0.0,0.0,0.0,0.0,0.000557,0.000759,0.001323,0.0,0.0,NaN,...,0.0,0.000044,0.000000,0.0,0.0,0.000847,-0.001697,0.000920,0.000705,0.0
2022-12-02,0.0,0.0,0.0,0.0,0.000557,0.000143,0.000367,0.0,0.0,NaN,...,0.0,0.000000,0.003327,0.0,0.0,0.000891,0.000000,0.001181,0.000320,0.0
2022-12-03,0.0,0.0,0.0,0.0,0.000557,0.000000,0.000000,0.0,0.0,NaN,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change().round(7)
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-12-01         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-12-02         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-12-03         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-12-01          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2022-12-02          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2022-12-03          0.0      0.0  ...     0.0    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-12-01            0.0   0.0   0.0        NaN      0.0    0.0  
2022-12-02            0.0   0.0   0.0        NaN      0.0    0.0  
2022-12-03            0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [72]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [73]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean().round(7)
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-12-01,0.000582,0.000034,0.000025,0.002109,0.000468,0.0,0.0,0.000011,0.000015,0.004050,...,0.000063,0.000627,0.0,0.000235,0.000046,0.0,0.0,0.0,0.000010,0.001232
2022-12-02,0.000436,0.000048,0.000020,0.001055,0.000234,0.0,0.0,0.000005,0.000008,0.002305,...,0.000031,0.000636,0.0,0.000117,0.000051,0.0,0.0,0.0,0.000005,0.000616
2022-12-03,0.000247,0.000039,0.000014,0.000527,0.000117,0.0,0.0,0.000003,0.000004,0.001300,...,0.000016,0.000318,0.0,0.000188,0.000042,0.0,0.0,0.0,0.000002,0.000308


In [74]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [75]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-12-01,0.000064,0.0,0.0,0.001603,0.000136,NaN,0.0,1.900000e-06,0.000007,0.002386,...,0.000058,0.0,0.0,0.000003,0.000035,0.0,NaN,0.0,5.000000e-07,0.000624
2022-12-02,0.000032,0.0,0.0,0.000801,0.000068,NaN,0.0,1.000000e-06,0.000004,0.002458,...,0.000029,0.0,0.0,0.000001,0.000017,0.0,NaN,0.0,2.000000e-07,0.000312
2022-12-03,0.000016,0.0,0.0,0.000401,0.000034,NaN,0.0,5.000000e-07,0.000002,0.001229,...,0.000015,0.0,0.0,0.000087,0.000020,0.0,NaN,0.0,1.000000e-07,0.000156


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [77]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean().round(7)
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-12-01,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-12-02,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-12-03,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078


In [78]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [79]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-12-01,0.000681,0.000168,0.000008,0.001408,0.000670,0.000765,0.000864,0.000741,0.000699,0.0,...,0.000007,0.000461,0.000036,0.000576,0.000453,0.000802,0.001067,0.000602,0.000753,0.000349
2022-12-02,0.000340,0.000084,0.000004,0.000704,0.000724,0.000828,0.000830,0.000948,0.000714,0.0,...,0.000003,0.000270,0.002093,0.000288,0.000418,0.000845,0.000534,0.000539,0.000751,0.000175
2022-12-03,0.000170,0.000042,0.000002,0.000352,0.000599,0.000414,0.000415,0.000763,0.000357,0.0,...,0.000002,0.000135,0.001046,0.000144,0.000209,0.000423,0.000267,0.000269,0.000376,0.000087


In [80]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [81]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-12-01      0.000915  0.000946  0.000433  0.000332  0.000578  0.000353   
2022-12-02      0.000458  0.000473  0.000217  0.000166  0.000289  0.000177   
2022-12-03      0.000229  0.000237  0.000108  0.000083  0.000144  0.000088   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2022-12-01      0.001029  0.000674  0.001141  0.004728  ...  0.000084   
2022-12-02      0.000515  0.000337  0.000570  0.002364  ...  0.000042   
2022-12-03      0.000257  0.000169  0.000285  0.001182  ...  0.000021   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2022-12-01      0.000398  0.000689  0.000112   0.000421  0.000244  0.000323   
2022-12-02      0.000199  0.000344  0.000056   0.000210  0.000122  0.000162   
2022-12-03      0.000099  0.000172  0.000028   0.000105  0.000061  0.000081   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2022-12-01      -0.503937  0.000284  0.000746  
2022-12-02      -0.503937  0.000142  0.000373  
2022-12-03      -0.503937  0.000071  0.000187  

[3 rows x 3336 columns]

In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [83]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-12-01,0.000256,1.000000e-07,0.0,0.000335,0.000490,0.000470,0.000885,0.000163,0.000006,NaN,...,1.000000e-06,0.000094,0.000032,0.000018,0.002016,0.000844,-0.001065,0.000840,0.000590,0.000457
2022-12-02,0.000128,1.000000e-07,0.0,0.000168,0.000524,0.000307,0.000626,0.000082,0.000003,NaN,...,5.000000e-07,0.000047,0.001680,0.000009,0.001008,0.000868,-0.000533,0.001011,0.000455,0.000228
2022-12-03,0.000064,0.000000e+00,0.0,0.000084,0.000540,0.000153,0.000313,0.000041,0.000001,NaN,...,2.000000e-07,0.000024,0.000840,0.000005,0.000504,0.000434,-0.000266,0.000505,0.000228,0.000114


In [84]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [85]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                \
Admin2           Autauga Baldwin Barbour Bibb    Blount Bullock Butler   
2022-12-01      0.000008     0.0     0.0  0.0  0.000969     0.0    0.0   
2022-12-02      0.000004     0.0     0.0  0.0  0.000485     0.0    0.0   
2022-12-03      0.000002     0.0     0.0  0.0  0.000242     0.0    0.0   

Province_State                            ...   Wyoming                       \
Admin2         Calhoun Chambers Cherokee  ...      Park Platte      Sheridan   
2022-12-01         0.0      0.0      0.0  ...  0.000833    0.0  1.000000e-07   
2022-12-02         0.0      0.0      0.0  ...  0.000417    0.0  0.000000e+00   
2022-12-03         0.0      0.0      0.0  ...  0.000208    0.0  0.000000e+00   

Province_State                                                      \
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie   
2022-12-01          0.0   0.002863   0.0   0.0       -1.0      0.0   
2022-12-02          0.0   0.001431   0.0   0.0       -1.0      0.0   
2022-12-03          0.0   0.000716   0.0   0.0       -1.0      0.0   

Province_State                
Admin2                Weston  
2022-12-01      4.000000e-07  
2022-12-02      2.000000e-07  
2022-12-03      1.000000e-07  

[3 rows x 3336 columns]

In [86]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [87]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

HTTPError: HTTP Error 403: Forbidden

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [ ]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [ ]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

In [ ]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

#### 